# Tutorial 2

## Make a scenario of Dantzig's Transport Problem using the IX modeling platform (ixmp)


In [ ]:
# load required packages 
import pandas as pd
import ixmp as ix

In [ ]:
# launch the IX modeling platform using the local default database                                                                                                                       
mp = ix.Platform(dbtype='HSQLDB')

In [ ]:
# details for loading an existing datastructure from the IX modeling platform                                                                                                                                                     
model = "canning problem"
scen = "standard"

mp.model_scenario_list()

In [ ]:
# load the datastructure
ds = mp.DataStructure(model, scen)

# create a new scenario by cloning the datastructure (without keeping the solution)
ds_scen = ds.clone(model, 'detroit', annotation='extend the Transport problem by a new city', keep_sol=False)

In [ ]:
# load the distance parameter
d = ds_scen.par("d")
print(d)                                                                                                                                   

In [ ]:
# show only the distances for connections from Seattle
print(d[d['i'] == "seattle"])

In [ ]:
# for faster access or more complex filtering,
# it may be easier to only load specific parameter elements using a dictionary
ele_filter = {}
ele_filter['i'] = ['seattle']
ele_filter['j'] = ['chicago', 'topeka']

d_filter = ds.par("d", ele_filter)

print(d_filter)

In [ ]:
# check out the datastructure to make changes
ds_scen.check_out()

In [ ]:
# reduce demand 
ds_scen.add_par('b', 'chicago', 200, 'cases')

# add a new city with demand and distances
ds_scen.add_set('j', 'detroit')
ds_scen.add_par('b', 'detroit', 150, 'cases')
ds_scen.add_par('d', ['seattle', 'detroit'], 1.7, 'cases')
ds_scen.add_par('d', ['san-diego', 'detroit'], 1.9, 'cases')

In [ ]:
# save changes to database
comment = "add new city 'detroit' with demand, reduce demand in 'chicago'"
ds_scen.commit(comment)                                                                                                       
ds_scen.set_as_default()

In [ ]:
# export the datastructure to GAMS gdx, solve it in GAMS,
# and import the solution back to the ix Modeling Platform
ds_scen.solve(model='transport_ixmp')

In [ ]:
# display the objective value of the solution
ds_scen.var("z")

In [ ]:
# display the quantities transported from canning plants to demand locations
ds_scen.var("x")

In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints
ds_scen.equ("demand")

In [ ]:
# close the connection of the platform instance to the local database
# this removes the 'locking files' so that local database files can be moved
mp.close_db()